In [ ]:
def get_startlist(URL, RACENAME):
        
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    title = soup.title.text
    if title.startswith("Page not found"):
        return []

    startlist_table = soup.find_all("table", class_ = "basic")[0]
    rows = startlist_table.find("tbody").find_all("tr")

    startlist = []

    for i, row in enumerate(rows):
        cols = row.find_all("td")
        startlist.append(cols[2].text.strip())

    startlist = [s.title() for s in startlist]        

    txt = OVERVIEW.to_html()

    for rider in startlist:
        to_replace = "<td>" + rider + "</td>"
        replacement = '<td class = "startlist">' + rider + "</td>"
        txt = txt.replace(to_replace, replacement)

    txt = """<html>
    <head>
    <style>
      body {
        font-family: 'Trebuchet MS', 'Lucida Sans Unicode', 'Lucida Grande', 'Lucida Sans', Arial, sans-serif;
      }

      table {
        font-size: 11px;
        border-collapse: collapse;
        width: 90%;
      }
      
      td, th {
        border: 1px solid #dddddd;
        text-align: left;
        padding: 8px;
      }
    .startlist {
    background-color: green;
    }
    </style>
    </head>
    <body>
    """ + txt + """

    </body>
    </html>"""

    print("DONE:", RACENAME)

    with open("../Løbsstartlister/" + RACENAME + ".md", "w") as file:
        file.write(txt)

In [ ]:
races = pd.read_csv("../data/races.csv")

NOW = datetime.datetime.now()
FROM = NOW + datetime.timedelta(days = -2)
TO = NOW + datetime.timedelta(days = 30)

RACES_TO_GET = races
RACES_TO_GET["RaceStart"] = pd.to_datetime(RACES_TO_GET["RaceStart"], format = "%Y-%m-%d %H:%M:00")
RACES_TO_GET["RaceEnd"] = pd.to_datetime(RACES_TO_GET["RaceEnd"], format = "%Y-%m-%d %H:%M:00")
RACES_TO_GET = races[(races["RaceEnd"] > FROM) & (races["RaceEnd"] < TO)]

for i, r in RACES_TO_GET.iterrows():
    racename = r['RaceName']
    racestart = str(r['RaceStart'])
    racestart = datetime.datetime.strptime(racestart, "%Y-%m-%d %H:%M:00")
    racestart = datetime.datetime.strftime(racestart, "%Y-%m-%d")
    racename = racestart + " " + racename
    url = r['RaceLink_PCS']
    url = url + "/startlist/alphabetical"
    get_startlist(url, racename)
    time.sleep(15)